In [9]:
#using Qwen generate summary

from datasets import load_dataset

import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import json
# Set OpenAI's API key and API base to use vLLM's API server.
from scipy.stats import kendalltau


prompt = "You are a helpful summary assistant. You can help users summarize news in two sentences." #two sentences


from openai import OpenAI          
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8880/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

#load dataset
from datasets import load_dataset

#/home/xbr/LLM/dataset/summary/sample_dataset
#cnndm_sample_0k5_1k5.json  newsroom_sample_0k5_1k5.json  xsum_sample_0k5_1k5.json

def clean_summary(summary):
     
    summary = summary.strip()
    summary = summary.replace('The news summary is: \"', '')
    summary = summary.split('\n')[0]
    candidate = summary
    complete_sentences = re.findall(r'[^.!?]*[.!?]', summary)
    # 将匹配到的完整句子连接成一个新的字符串
    summary = ''.join(complete_sentences)
    if(summary==''):
        summary = candidate
    
    return summary
    

#generate qwen summary
# data = json.load(open('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average.json'))
# data = json.load(open('/home/xbr/LLM/summary_benchmark/dataset/sample_500/cnndm_sample_500_0k5_1k5.jsonl'))
files = ['cnndm']
for f in files:
    # data = json.load(open('/home/xbr/LLM/summary_benchmark/dataset/sample_500/'+f+'_sample_500_0k5_1k5.jsonl'))
    
    for i in range(1):
        # news = data[i]['article']
        news = "(CNN)The outlines of a nuclear deal with Iran are in place. Unfortunately, it seems like too many in President Barack Obama's administration have forgotten that the only reason this terrorist-supporting state came to the negotiating table in the first place was because of tough sanctions imposed by the U.S. Congress. Indeed, the reality is that President Obama is giving up enormous leverage in his nuclear deal with Iran -- and I worry we will lose it for good. Bleeding money, and faced with falling oil prices, Supreme Leader Ayatollah Khamenei gave his government rare permission to bargain with the \"Great Satan\" -- the United States. But just as U.S. and European sanctions were forcing Iran to the nuclear crossroads, President Obama has given Tehran an easy exit. For Khamenei, the \"framework\" announced last week looks like a win-win: He gets to keep his nuclear infrastructure, and in return gets billions of dollars in sanctions relief. Congress offered a better strategy when the top Democrat on the Foreign Affairs Committee, Eliot Engel, and I introduced a bill to hit Tehran with its toughest sanctions yet. Unfortunately, this bill -- which passed the House in a 400-20 vote -- was blocked in the Senate last year, despite the fact that it would have sharpened the Ayatollah's choice: Dismantle your nuclear weapons program or see your economy collapse. President Obama once had a tougher line, when in 2012 he said: \"The deal we'll accept is they end their nuclear program. It's very straightforward.\" But the framework announced last week does nothing of the sort. Negotiated between Iran and the United States, Russia, China, Britain, France and Germany, the framework concedes that Iran can maintain \"a mutually defined enrichment program,\" operate thousands of centrifuges, and continue its research and development of nuclear technologies. The deal currently on the table would hand Tehran billions of previously sanctioned funds, filling the coffers of the world's biggest state sponsor of terrorism, with strongholds in Syria, Iraq, Yemen and Lebanon. Meanwhile, the strictest restrictions on Iran's enrichment will expire in only 10 years, despite the President receiving a letter from 367 Members of Congress -- both Democrats and Republicans -- in which we insisted that \"verifiable constraints on Iran's nuclear program must last for decades.\" The President admitted as much when he conceded that \"in year 13, 14, 15, they have advanced centrifuges that enrich uranium fairly rapidly, and at that point the breakout times would have shrunk almost down to zero.\" But as bad as these concessions are, the most concerning aspect of the April 2 deal is that it lacks tough safeguards to stop Iran from cheating. The key question is this: Will the inspectors at the International Atomic Energy Agency be allowed to inspect these military sites without warning? Because if the IAEA cannot conduct \"anytime, anywhere\" inspections, Iran will be able to \"sneak out\" to a bomb. It has been done before. Remember, in 1994, when President Bill Clinton told us he had struck a deal with North Korea that would \"make the United States, the Korean Peninsula, and the world safer\"? President Clinton sounded a little too much like the current Secretary of State John Kerry, when he promised that the North Korea agreement \"does not rely on trust\" and that \"compliance will be certified by the International Atomic Energy Agency.\" Twelve years after these assurances, North Korea detonated its first nuclear bomb. Iran could easily do the same. The best predictor of its future behavior is its past behavior -- between 2004 and 2009, the Iranian government built a huge centrifuge facility named Fordo under a mountain deep in the Iranian desert. Luckily for the world, Western intelligence agencies discovered Tehran's deception. But we cannot rely on such luck in the future, particularly when Iran still hasn't come clean about its history of secret weapons development and is still dodging basic questions from the IAEA. Let's not forget the other things Iran has been doing while its diplomats have been bargaining with the U.S. and its partners. While Iran was showing its friendly new face to the world, it has simultaneously been helping Syrian dictator Bashar al Assad kill his own people, training and funding the terrorist group Hezbollah, which aims to annihilate Israel, and supporting the Houthis, who started a civil war and overthrew the government in Yemen -- one of America's more reliable counterterrorism partners in the region. If President Obama is going to hand over billions of dollars to a regime that behaves like this, run by a man who publicly declares: \"Death to America,\" it has to be a better deal. The framework we have before us keeps Iran's nuclear door well and truly open."
        chat_response = client.chat.completions.create(
            model="Qwen/Qwen2-72B-Instruct",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": "Please summarize the news in two sentences.\nNews: "+news+'\nSummary: '}, #two sentences
                
            ],
            temperature=0,
            # max_tokens=100,
        )
        
        res = chat_response.choices[0].message.content
        print(res)   
        res = clean_summary(res)
        print(res)    
        # data[i]['qwen_reference_summary'] = res
        
    #save to json
    #./filter_annotations_summeval_llama2_summary.jsonl
    #/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average_with_yi34b.json
    # with open('./data/likert_evaluation_results_cnndm_average_with_qwen32b.jsonl', 'w') as f:
    # with open(f+'_sample_500_0k5_1k5_qwen2_summary_100_len_limit.jsonl', 'w') as f:
    #     json.dump(data, f, indent=4)


The summary is as follows:
The author criticizes the nuclear deal with Iran, arguing that the US is giving up leverage by easing sanctions before Iran dismantles its nuclear program. The deal allows Iran to maintain a nuclear program, receive billions in sanctions relief, and potentially cheat without strict inspections, while restrictions on enrichment expire in 10 years, risking a rapid breakout capability. The author draws parallels to North Korea's deception and warns of Iran's past behavior, ongoing support for terrorism, and hostility towards the US and Israel.
The summary is as follows:
